In [108]:
import pandas as pd

df1=pd.read_csv("https://raw.githubusercontent.com/statzenthusiast921/political-emails-analysis/main/main/data/emails1_df.csv")
df2=pd.read_csv("https://raw.githubusercontent.com/statzenthusiast921/political-emails-analysis/main/main/data/emails2_df.csv")
df3=pd.read_csv("https://raw.githubusercontent.com/statzenthusiast921/political-emails-analysis/main/main/data/emails3_df.csv")
df4=pd.read_csv("https://raw.githubusercontent.com/statzenthusiast921/political-emails-analysis/main/main/data/emails4_df.csv")

print(df1.shape)
print(df2.shape)
print(df3.shape)
print(df4.shape)

(20000, 9)
(20000, 9)
(20000, 9)
(23316, 9)


In [109]:
#Stack 4 dataframes together
df = pd.concat([df1, df2, df3, df4], ignore_index=True, axis=0)
#Delete first row
df = df[1:]
#Delete first column
df = df.iloc[: , 1:]
#Preview dataframe
df.head(2)

,sender,subject,date,body,party,country,locality,office
1,JusticeDemocrats.com <[email protected]>,Critical preparations,"July 3, 2019 11:28 PM",[1]Justice Democrats\n\r\n John -- \n\r\n ...,Political Party:\nDemocratic,Country:\n\nUnited States,State/Locality:\n\t\t\t\t\t\t\t\t\t\t\t\t\tn/a,Office:\n\t\t\t\t\t\t\t\t\t\t\t\t\tn/a
2,Ana Maria Archila <[email protected]>,TAKE ACTION for Freedom: #CloseTheCamps,"July 3, 2019 11:31 PM","Dear friend,\n\r\n \n\r\nWe have all seen the ...",Political Party:\n\t\t\t\t\t\t\t\t\t\t\t\t\tn/a,Country:\n\nUnited States,State/Locality:\n\t\t\t\t\t\t\t\t\t\t\t\t\tn/a,Office:\n\t\t\t\t\t\t\t\t\t\t\t\t\tn/a


In [110]:
#Clean up columns - change data type
df['sender'] = df['sender'].astype('string')
df['subject'] = df['subject'].astype('string')
df['date'] = df['date'].astype('string')
df['body'] = df['body'].astype('string')
df['party'] = df['party'].astype('string')
df['country'] = df['country'].astype('string')
df['locality'] = df['locality'].astype('string')
df.dtypes

sender      string
subject     string
date        string
body        string
party       string
country     string
locality    string
office      object
dtype: object

In [111]:
import re 

df['party'] = df['party'].replace('Political Party:','',regex=True)
df['country'] = df['country'].replace('Country:','',regex=True)
df['locality'] = df['locality'].replace('State/Locality:','',regex=True)
df['office'] = df['office'].replace('Office:','',regex=True)
df = df.replace('\n','',regex=True)
df = df.replace('\t','',regex=True)
df['sender'] = df['sender'].replace('<','',regex=True)
df['sender'] = df['sender'].replace('>','',regex=True)

def remove_brackets_contents(text):
    pattern = r"\[.*?\]"
    return re.sub(pattern, " ", text)


def remove_special_characters(text):
    # define the pattern to keep
    pat = r'[^a-zA-z0-9.,!?/:;\"\'\s]' 
    return re.sub(pat, ' ', text)

df.loc[:,'sender'] = df['sender'].apply(func = remove_brackets_contents)


df.head(2)

,sender,subject,date,body,party,country,locality,office
1,JusticeDemocrats.com,Critical preparations,"July 3, 2019 11:28 PM",[1]Justice Democrats John -- Last month...,Democratic,United States,n/a,n/a
2,Ana Maria Archila,TAKE ACTION for Freedom: #CloseTheCamps,"July 3, 2019 11:31 PM","Dear friend, We have all seen the images and...",n/a,United States,n/a,n/a


In [112]:
#Cleaning up the email body column
import warnings
warnings.filterwarnings('ignore')

df['body'] = df['body'].replace('\n','',regex=True)
df['body'] = df['body'].replace('\r','',regex=True)
df.loc[:,'body'] = df['body'].apply(func = remove_brackets_contents)
df.loc[:,'body'] = df['body'].apply(func = remove_special_characters)


df['body'] = df['body'].replace('   ',' ',regex=True)
df['body'] = df['body'].replace('    ',' ',regex=True)
df['body'] = df['body'].replace('     ',' ',regex=True)

df['body'] = df['body'].replace(']',' ',regex=True)
df['body'] = df['body'].replace('\xa0',' ',regex=True)
df['body'] = df['body'].replace('almostdaily','almost daily',regex=True)


#Remove all the text that follows these sentences
df['body'] = df['body'].str.replace("(If you need to remove yourself from our email).*","")
df['body'] = df['body'].str.replace("(Email is the most important way).*","")
df['body'] = df['body'].str.replace("(This email was sent to).*","")
df['body'] = df['body'].str.replace("(Thank you for speaking).*","")
df['body'] = df['body'].str.replace("(All Rights Reserved).*","")
df['body'] = df['body'].str.replace("(mailto:).*","")
df['body'] = df['body'].str.replace("(Thank you for choosing to fight).*","")
df['body'] = df['body'].str.replace("(Paid for by).*","")
df['body'] = df['body'].str.replace("(We look forward to hearing from you).*","")
df['body'] = df['body'].str.replace("(PAID FOR BY).*","")
df['body'] = df['body'].str.replace("(Thanks for your).*","")
df['body'] = df['body'].str.replace("(All rights reserved).*","")
df['body'] = df['body'].str.replace("(unsubscribe).*","")
df['body'] = df['body'].str.replace("(Sincerely).*","")
df['body'] = df['body'].str.replace("(Facebook).*","")
df['body'] = df['body'].str.replace("(Twitter).*","")
df['body'] = df['body'].str.replace("(Please, sign your name).*","")
df['body'] = df['body'].str.replace("(CONTRIBUTE).*","")
df['body'] = df['body'].str.replace("(Contribute).*","")

In [113]:
df[['split1','split2','split3','split4','split5']] = df['date'].str.split(' ',expand=True)
df['date'] = df['split1'] + ' ' + df['split2'] + ' ' + df['split3']
df['time'] = df['split4'] + ' ' + df['split5']

del df['split1'], df['split2'], df['split3'], df['split4'], df['split5']
print(df['date'][1])
print(df['time'][1])

July 3, 2019
11:28 PM


In [114]:
df.head(2)

,sender,subject,date,body,party,country,locality,office,time
1,JusticeDemocrats.com,Critical preparations,"July 3, 2019","Justice Democrats John Last month, we neede...",Democratic,United States,n/a,n/a,11:28 PM
2,Ana Maria Archila,TAKE ACTION for Freedom: #CloseTheCamps,"July 3, 2019","Dear friend, We have all seen the images and r...",n/a,United States,n/a,n/a,11:31 PM


In [115]:
import numpy as np

df['AM_PM'] = df['time'].str[-2:]
df['Hour'] = df['time'].str[:2]
df['Hour'] = np.where(df['Hour'].str[1:]==":",df['Hour'].str[:1],df['Hour'])
df['Hour'] = df['Hour'].astype(int)

df['Hour_Mil'] = np.where(
    df['AM_PM'] == 'PM', df['Hour'] + 12, np.where(
    (df['AM_PM'] == 'AM') & (df['Hour'] == 12), df['Hour'] - 12, df['Hour'])) 


df.head(2)

,sender,subject,date,body,party,country,locality,office,time,AM_PM,Hour,Hour_Mil
1,JusticeDemocrats.com,Critical preparations,"July 3, 2019","Justice Democrats John Last month, we neede...",Democratic,United States,n/a,n/a,11:28 PM,PM,11,23
2,Ana Maria Archila,TAKE ACTION for Freedom: #CloseTheCamps,"July 3, 2019","Dear friend, We have all seen the images and r...",n/a,United States,n/a,n/a,11:31 PM,PM,11,23


In [116]:
#Grab the day, month, year
df[['month','day','year']] = df['date'].str.split(' ',expand=True)
df['day'] = df['day'].str[:1]


df['month_num'] = np.where(
    df['month'] == 'January', 1, np.where(
    df['month'] == 'February', 2, np.where(
    df['month'] == 'March', 3, np.where(
    df['month'] == 'April', 4, np.where(
    df['month'] == 'May', 5, np.where(
    df['month'] == 'June', 6, np.where(
    df['month'] == 'July', 7, np.where(
    df['month'] == 'August', 8, np.where(
    df['month'] == 'September', 9, np.where(
    df['month'] == 'October', 10, np.where(
    df['month'] == 'November', 11, 12
)))))))))))



df.head(2)

,sender,subject,date,body,party,country,locality,office,time,AM_PM,Hour,Hour_Mil,month,day,year,month_num
1,JusticeDemocrats.com,Critical preparations,"July 3, 2019","Justice Democrats John Last month, we neede...",Democratic,United States,n/a,n/a,11:28 PM,PM,11,23,July,3,2019,7
2,Ana Maria Archila,TAKE ACTION for Freedom: #CloseTheCamps,"July 3, 2019","Dear friend, We have all seen the images and r...",n/a,United States,n/a,n/a,11:31 PM,PM,11,23,July,3,2019,7


In [117]:
df_dom = df[df['country']=="United States"]
df_for = df[df['country']!="United States"]

print(df_dom.shape)
print(df_for.shape)

(74148, 16)
(9167, 16)


In [118]:
#When are emails sent?
import plotly.express as px
new_df = df_dom.groupby(['Hour_Mil']).sum().reset_index()
new_df.rename(columns={ new_df.columns[1]: "Count" }, inplace = True)

fig = px.line(new_df, x="Hour_Mil", y="Count", title='US Emails Sent by Hour')
fig.show()

In [119]:
month1_emails = df_dom[df_dom['month_num']==1]
month2_emails = df_dom[df_dom['month_num']==2]
month3_emails = df_dom[df_dom['month_num']==3]
month4_emails = df_dom[df_dom['month_num']==4]
month5_emails = df_dom[df_dom['month_num']==5]
month6_emails = df_dom[df_dom['month_num']==6]
month7_emails = df_dom[df_dom['month_num']==7]
month8_emails = df_dom[df_dom['month_num']==8]
month9_emails = df_dom[df_dom['month_num']==9]
month10_emails = df_dom[df_dom['month_num']==10]
month11_emails = df_dom[df_dom['month_num']==11]
month12_emails = df_dom[df_dom['month_num']==12]

m1 = month1_emails.groupby(['Hour_Mil']).sum().reset_index()
m2 = month2_emails.groupby(['Hour_Mil']).sum().reset_index()
m3 = month3_emails.groupby(['Hour_Mil']).sum().reset_index()
m4 = month4_emails.groupby(['Hour_Mil']).sum().reset_index()
m5 = month5_emails.groupby(['Hour_Mil']).sum().reset_index()
m6 = month6_emails.groupby(['Hour_Mil']).sum().reset_index()
m7 = month7_emails.groupby(['Hour_Mil']).sum().reset_index()
m8 = month8_emails.groupby(['Hour_Mil']).sum().reset_index()
m9 = month9_emails.groupby(['Hour_Mil']).sum().reset_index()
m10 = month10_emails.groupby(['Hour_Mil']).sum().reset_index()
m11 = month11_emails.groupby(['Hour_Mil']).sum().reset_index()
m12 = month12_emails.groupby(['Hour_Mil']).sum().reset_index()

m1.rename(columns={ m1.columns[1]: "Count" }, inplace = True)
m2.rename(columns={ m2.columns[1]: "Count" }, inplace = True)
m3.rename(columns={ m3.columns[1]: "Count" }, inplace = True)
m4.rename(columns={ m4.columns[1]: "Count" }, inplace = True)
m5.rename(columns={ m5.columns[1]: "Count" }, inplace = True)
m6.rename(columns={ m6.columns[1]: "Count" }, inplace = True)
m7.rename(columns={ m7.columns[1]: "Count" }, inplace = True)
m8.rename(columns={ m8.columns[1]: "Count" }, inplace = True)
m9.rename(columns={ m9.columns[1]: "Count" }, inplace = True)
m10.rename(columns={ m10.columns[1]: "Count" }, inplace = True)
m11.rename(columns={ m11.columns[1]: "Count" }, inplace = True)
m12.rename(columns={ m12.columns[1]: "Count" }, inplace = True)


In [120]:
import dash_core_components as dcc
import plotly.express as px
import plotly.subplots as sp
import plotly.graph_objects as go
from plotly.subplots import make_subplots

#Make subplots to display different slices of data all together on one plot
fig = make_subplots(rows=2, cols=2, start_cell="top-left")

fig.add_trace(go.Line(x=m9['Hour_Mil'], y=m9['Count'], name='Month 9'),row=1, col=1)
fig.add_trace(go.Line(x=m10['Hour_Mil'], y=m10['Count'], name='Month 10'),row=1, col=2)
fig.add_trace(go.Line(x=m11['Hour_Mil'], y=m11['Count'], name='Month 11'),row=2, col=1)
fig.add_trace(go.Line(x=m12['Hour_Mil'], y=m12['Count'], name='Month 12'),row=2, col=2)

fig.show()


In [121]:
df_dom['State'] = np.where(df_dom['locality'].isin([
    'Alabama', 'Alaska', 'Arizona', 'Arkansas', 'California', 
    'Colorado', 'Connecticut', 'Delaware', 'District of Columbia', 
    'Florida', 'Georgia', 'Hawaii', 'Idaho', 'Illinois', 
    'Indiana', 'Iowa', 'Kansas', 'Kentucky', 'Louisiana', 
    'Maine', 'Maryland', 'Massachusetts', 'Michigan', 'Minnesota',
    'Mississippi', 'Missouri', 'Montana', 'Nebraska', 'Nevada', 
    'New Hampshire', 'New Jersey', 'New Mexico', 'New York', 'North Carolina', 
    'North Dakota', 'Ohio', 'Oklahoma', 'Oregon', 'Pennsylvania',
    'Rhode Island', 'South Carolina', 'South Dakota', 'Tennessee', 
    'Texas', 'Utah', 'Vermont', 'Virginia', 'Washington', 'West Virginia', 
    'Wisconsin', 'Wyoming'
]),1,0)
df_dom['State'].value_counts()

0    46324
1    27824
Name: State, dtype: int64

In [122]:
import pandas as pd
test = df_dom[df_dom['State']==1]
test = test[['locality']]

# Get State Crosswalk From Generic Website
crosswalk = 'http://app02.clerk.org/menu/ccis/Help/CCIS%20Codes/state_codes.html'
states = pd.read_html(crosswalk)[0]

# Demo Crosswalking State Name to State Abbreviation
test['ST'] = test['locality'].map(dict(zip(states['Description'], 
                                         states['Code'])))

test.head()

,locality,ST
3,Ohio,OH
4,Texas,TX
5,Kentucky,KY
8,Washington,WA
9,California,CA


In [123]:
map_test = pd.DataFrame(test['ST'].value_counts()).reset_index()

map_test.rename(
    columns={
        map_test.columns[0]: "ST",
        map_test.columns[1]: "Count"

    }, inplace = True)


fig = px.choropleth(
    map_test,
    locations=map_test['ST'], 
    locationmode="USA-states", 
    color="Count",    
    scope="usa",
)
fig.show()